## P-9.53 Implement the in-place heap-sort algorithm. Experimentally compare its running time with that of the standard heap-sort that is not in-place.

### In-Place Heap Sort

In [ ]:
def heap(arr, n, i):
    largest = i  # Initialize largest as root
    left = 2 * i + 1  # left = 2*i + 1
    right = 2 * i + 2  # right = 2*i + 2

    # If left child is larger than root
    if left < n and arr[left] > arr[largest]:
        largest = left

    # If right child is larger than largest so far
    if right < n and arr[right] > arr[largest]:
        largest = right

    # If largest is not root
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]  # swap

        # Recursively heap the affected sub-tree
        heap(arr, n, largest)

def heap_sort_in_place(arr):
    n = len(arr)

    # Build a maxheap
    for i in range(n // 2 - 1, -1, -1):
        heap(arr, n, i)

    # One by one extract elements from heap
    for i in range(n - 1, 0, -1):
        arr[i], arr[0] = arr[0], arr[i]  # swap
        heap(arr, i, 0)

### Standard Heap Sort

In [ ]:
def standard_heap_sort(arr):
    n = len(arr)
    heap1 = []

    # Build a max heap
    for value in arr:
        heap1.append(value)
        # Maintain the heap property
        i = len(heap1) - 1
        while i > 0:
            parent = (i - 1) // 2
            if heap1[i] > heap1[parent]:
                heap1[i], heap1[parent] = heap1[parent], heap1[i]
                i = parent
            else:
                break

    # Extract elements from the heap
    sorted_arr = []
    while heap1:
        # Move the largest element to the end of the sorted array
        heap1[0], heap1[-1] = heap1[-1], heap1[0]
        sorted_arr.append(heap1.pop())
        # Re-heap
        heap(heap1, len(heap1), 0)

    return sorted_arr

### Time Comparison

In [ ]:
import time
import random

# Generate a random list of integers
def generate_random_list(size):
    return [random.randint(1, 10000) for _ in range(size)]

# Compare running times
def compare_heap_sorts(size):
    random_list = generate_random_list(size)

    # In-place heap sort
    arr_in_place = random_list.copy()
    start_time = time.time()
    heap_sort_in_place(arr_in_place)
    in_place_time = time.time() - start_time

    # Standard heap sort
    arr_standard = random_list.copy()
    start_time = time.time()
    sorted_arr = standard_heap_sort(arr_standard)
    standard_time = time.time() - start_time

    print(f"Size: {size}, In-Place Time: {in_place_time:.6f}s, Standard Time: {standard_time:.6f}s")

# Run the comparison for different sizes
for size in [1000, 5000, 10000, 50000, 100000, 500000, 1000000]:
    compare_heap_sorts(size)

Size: 1000, In-Place Time: 0.004796s, Standard Time: 0.004987s
Size: 5000, In-Place Time: 0.038514s, Standard Time: 0.034481s
Size: 10000, In-Place Time: 0.066438s, Standard Time: 0.068676s
Size: 50000, In-Place Time: 0.457390s, Standard Time: 0.441698s
Size: 100000, In-Place Time: 1.509872s, Standard Time: 1.691835s
Size: 500000, In-Place Time: 6.222526s, Standard Time: 7.484966s
Size: 1000000, In-Place Time: 14.665295s, Standard Time: 14.899644s


##P-9.53 Write a program that can process a sequence of stock buy and sell orders as described in Exercise C-9.50.

In [2]:
import heapq


class StockOrderSystem:
    def __init__(self):
        self.buy_orders = []  # Max-heap for buy orders (using negative prices)
        self.sell_orders = []  # Min-heap for sell orders
        self.trade_log = []  # List to store executed trades

    def buy(self, shares, price):
        # Add the buy order to the max-heap (using negative price for max behavior)
        heapq.heappush(self.buy_orders, (-price, shares))
        self.process_orders()

    def sell(self, shares, price):
        # Add the sell order to the min-heap
        heapq.heappush(self.sell_orders, (price, shares))
        self.process_orders()

    def process_orders(self):
        while self.buy_orders and self.sell_orders:
            # Get the highest buy order and the lowest sell order
            buy_price, buy_shares = self.buy_orders[0]
            sell_price, sell_shares = self.sell_orders[0]

            # Convert buy_price back to positive
            buy_price = -buy_price

            # Display the current buy and sell orders
            print(f"Current Buy Order: {buy_shares} shares at ₱{buy_price} each")
            print(f"Current Sell Order: {sell_shares} shares at ₱{sell_price} each")

            # Check if the orders can be processed
            if sell_price <= buy_price:
                # Determine the number of shares to process
                shares_to_trade = min(buy_shares, sell_shares)

                # Record the trade in the trade log
                self.trade_log.append({
                    'shares': shares_to_trade,
                    'price': sell_price,
                    'type': 'buy' if sell_price <= buy_price else 'sell'
                })

                # Print the transaction for both buy and sell
                print(f"Processed: Buy {shares_to_trade} shares at ₱{sell_price} each")
                print(f"Processed: Sell {shares_to_trade} shares at ₱{sell_price} each")

                # Update the shares
                if buy_shares > shares_to_trade:
                    self.buy_orders[0] = (-buy_price, buy_shares - shares_to_trade)
                else:
                    heapq.heappop(self.buy_orders)

                if sell_shares > shares_to_trade:
                    self.sell_orders[0] = (sell_price, sell_shares - shares_to_trade)
                else:
                    heapq.heappop(self.sell_orders)
            else:
                break  # No more orders can be processed

    def print_trade_log(self):
        print("\nTrade Log:")
        for trade in self.trade_log:
            print(f"Traded {trade['shares']} shares at ₱{trade['price']} each.")


# Example usage
if __name__ == "__main__":
    system = StockOrderSystem()

    # Sample orders
    system.buy(100, 10)  # Buy 100 shares at ₱10 each
    system.sell(100, 9)  # Sell 100 shares at ₱9 each
    system.sell(50, 10)  # Sell 50 shares at ₱10 each
    system.buy(50, 10)  # Buy 50 shares at ₱10 each

    # Print the trade log
    system.print_trade_log()

Current Buy Order: 100 shares at ₱10 each
Current Sell Order: 100 shares at ₱9 each
Processed: Buy 100 shares at ₱9 each
Processed: Sell 100 shares at ₱9 each
Current Buy Order: 50 shares at ₱10 each
Current Sell Order: 50 shares at ₱10 each
Processed: Buy 50 shares at ₱10 each
Processed: Sell 50 shares at ₱10 each

Trade Log:
Traded 100 shares at ₱9 each.
Traded 50 shares at ₱10 each.
